## Import Dependencies

In [34]:
import pandas as pd
import numpy as np
import psycopg2
import sqlalchemy
from pymongo import MongoClient
from config import path

## Inital Data Import and cursory examination

In [7]:
salary_data = pd.read_csv('Data/ds_salaries.csv', index_col=[0])
realtor_data = pd.read_csv('Data/realtor-data.csv')

In [8]:
salary_data

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L
...,...,...,...,...,...,...,...,...,...,...,...
602,2022,SE,FT,Data Engineer,154000,USD,154000,US,100,US,M
603,2022,SE,FT,Data Engineer,126000,USD,126000,US,100,US,M
604,2022,SE,FT,Data Analyst,129000,USD,129000,US,0,US,M
605,2022,SE,FT,Data Analyst,150000,USD,150000,US,100,US,M


In [9]:
realtor_data

,status,price,bed,bath,acre_lot,full_address,street,city,state,zip_code,house_size,sold_date
0,for_sale,105000.0,3.0,2.0,0.12,"Sector Yahuecas Titulo # V84, Adjuntas, PR, 00601",Sector Yahuecas Titulo # V84,Adjuntas,Puerto Rico,601.0,920.0,NaN
1,for_sale,80000.0,4.0,2.0,0.08,"Km 78 9 Carr # 135, Adjuntas, PR, 00601",Km 78 9 Carr # 135,Adjuntas,Puerto Rico,601.0,1527.0,NaN
2,for_sale,67000.0,2.0,1.0,0.15,"556G 556-G 16 St, Juana Diaz, PR, 00795",556G 556-G 16 St,Juana Diaz,Puerto Rico,795.0,748.0,NaN
3,for_sale,145000.0,4.0,2.0,0.10,"R5 Comunidad El Paraso Calle De Oro R-5 Ponce,...",R5 Comunidad El Paraso Calle De Oro R-5 Ponce,Ponce,Puerto Rico,731.0,1800.0,NaN
4,for_sale,65000.0,6.0,2.0,0.05,"14 Navarro, Mayaguez, PR, 00680",14 Navarro,Mayaguez,Puerto Rico,680.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
923154,for_sale,445000.0,1.0,2.0,0.99,"1008 King St, Chappaqua, NY, 10514",1008 King St,Chappaqua,New York,10514.0,1052.0,2011-05-09
923155,for_sale,418000.0,4.0,2.0,0.40,"3 Elmwood Dr, Monroe, NY, 10950",3 Elmwood Dr,Monroe,New York,10950.0,1650.0,2015-07-21
923156,for_sale,469000.0,4.0,2.0,0.18,"13 N Conger Ave, Congers, NY, 10920",13 N Conger Ave,Congers,New York,10920.0,2123.0,NaN
923157,for_sale,825000.0,5.0,5.0,0.79,"7 Miller Rd, Valley Cottage, NY, 10989",7 Miller Rd,Valley Cottage,New York,10989.0,3775.0,2010-06-02


# Filter Dataframe by state to both limit size and make relatable to people in region around Penn

In [30]:
realtor_pa = realtor_data.loc[realtor_data['state'] == 'Pennsylvania'].reset_index(drop=True)

In [31]:
realtor_pa

,status,price,bed,bath,acre_lot,full_address,street,city,state,zip_code,house_size,sold_date
0,for_sale,1470000.0,5.0,5.0,0.92,"111 Apple Blossom Ln, Newtown Square, PA, 19073",111 Apple Blossom Ln,Newtown Square,Pennsylvania,19073.0,4953.0,NaN
1,for_sale,1445000.0,5.0,5.0,0.92,"110 Apple Blossom Ln, Newtown Square, PA, 19073",110 Apple Blossom Ln,Newtown Square,Pennsylvania,19073.0,4953.0,NaN
2,for_sale,1445000.0,5.0,5.0,NaN,"112 Apple Blossom Ln, Newtown Square, PA, 19073",112 Apple Blossom Ln,Newtown Square,Pennsylvania,19073.0,NaN,NaN
3,for_sale,1420000.0,5.0,5.0,0.92,"100 Apple Blossom Ln, Newtown Square, PA, 19073",100 Apple Blossom Ln,Newtown Square,Pennsylvania,19073.0,4953.0,NaN
4,for_sale,1445000.0,5.0,5.0,0.92,"108 Apple Blossom Ln, Newtown Square, PA, 19073",108 Apple Blossom Ln,Newtown Square,Pennsylvania,19073.0,4953.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
17143,for_sale,429900.0,2.0,2.0,NaN,"5 Aspen, New Hope, PA, 18938",5 Aspen,New Hope,Pennsylvania,18938.0,1564.0,2019-09-27
17144,for_sale,1299900.0,3.0,4.0,0.78,"2472 River Rd, New Hope, PA, 18938",2472 River Rd,New Hope,Pennsylvania,18938.0,2780.0,2020-11-19
17145,for_sale,1695000.0,4.0,5.0,0.11,"39 New St, New Hope, PA, 18938",39 New St,New Hope,Pennsylvania,18938.0,3200.0,2019-02-21
17146,for_sale,2250000.0,4.0,5.0,1.28,"3044 River Rd, New Hope, PA, 18938",3044 River Rd,New Hope,Pennsylvania,18938.0,3913.0,1997-04-10


# Filter realtor and salary data to relevent columns

In [45]:
realtor_columns = realtor_pa[['price', 'city']]

In [46]:
realtor_columns.describe()

,price
count,1.714800e+04
mean,4.245694e+05
std,7.111105e+05
min,4.000000e+03
25%,1.850000e+05
50%,2.850000e+05
75%,4.502500e+05
max,3.400000e+07


In [29]:
new_salary = salary_data.loc[(salary_data['experience_level'] == 'EN') | (salary_data['experience_level'] == 'MI')].reset_index(drop=True)
new_salary

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
2,2020,EN,FT,Data Analyst,72000,USD,72000,US,100,US,L
3,2020,MI,FT,Data Scientist,11000000,HUF,35735,HU,50,HU,L
4,2020,MI,FT,Business Data Analyst,135000,USD,135000,US,100,US,L
...,...,...,...,...,...,...,...,...,...,...,...
296,2022,MI,FT,Data Scientist,160000,USD,160000,US,100,US,M
297,2022,MI,FT,Data Scientist,130000,USD,130000,US,100,US,M
298,2022,EN,FT,Data Analyst,67000,USD,67000,CA,0,CA,M
299,2022,EN,FT,Data Analyst,52000,USD,52000,CA,0,CA,M


In [33]:
final_salary = new_salary[['salary_in_usd', 'experience_level', 'job_title']]
final_salary

,salary_in_usd,experience_level,job_title
0,79833,MI,Data Scientist
1,20000,MI,Product Data Analyst
2,72000,EN,Data Analyst
3,35735,MI,Data Scientist
4,135000,MI,Business Data Analyst
...,...,...,...
296,160000,MI,Data Scientist
297,130000,MI,Data Scientist
298,67000,EN,Data Analyst
299,52000,EN,Data Analyst


## Connect to mongodb and add dataframes in

In [35]:
client = MongoClient(path)

In [36]:
db = client.data_money
salary = db.salary

In [39]:
salary_dict = final_salary.to_dict("records")
salary.insert_many(salary_dict) 

,_id,salary_in_usd,experience_level,job_title
0,631fc601a3097e7ff7d417b7,79833,MI,Data Scientist
1,631fc601a3097e7ff7d417b8,20000,MI,Product Data Analyst
2,631fc601a3097e7ff7d417b9,72000,EN,Data Analyst
3,631fc601a3097e7ff7d417ba,35735,MI,Data Scientist
4,631fc601a3097e7ff7d417bb,135000,MI,Business Data Analyst


In [48]:
salary_mongo = pd.DataFrame(salary.find())
salary_mongo.head()

,_id,salary_in_usd,experience_level,job_title
0,631fc601a3097e7ff7d417b7,79833,MI,Data Scientist
1,631fc601a3097e7ff7d417b8,20000,MI,Product Data Analyst
2,631fc601a3097e7ff7d417b9,72000,EN,Data Analyst
3,631fc601a3097e7ff7d417ba,35735,MI,Data Scientist
4,631fc601a3097e7ff7d417bb,135000,MI,Business Data Analyst


In [40]:
real_estate = db.real_estate

In [47]:
realtor_dict = realtor_columns.to_dict('records')
real_estate.insert_many(realtor_dict)

,_id,price,city
0,631fc70da3097e7ff7d418e4,1470000.0,Newtown Square
1,631fc70da3097e7ff7d418e5,1445000.0,Newtown Square
2,631fc70da3097e7ff7d418e6,1445000.0,Newtown Square
3,631fc70da3097e7ff7d418e7,1420000.0,Newtown Square
4,631fc70da3097e7ff7d418e8,1445000.0,Newtown Square


## Pull Information from mongodb and use it for minor analysis

In [49]:
real_estate_mongo = pd.DataFrame(real_estate.find())
real_estate_mongo.head()

,_id,price,city
0,631fc70da3097e7ff7d418e4,1470000.0,Newtown Square
1,631fc70da3097e7ff7d418e5,1445000.0,Newtown Square
2,631fc70da3097e7ff7d418e6,1445000.0,Newtown Square
3,631fc70da3097e7ff7d418e7,1420000.0,Newtown Square
4,631fc70da3097e7ff7d418e8,1445000.0,Newtown Square


In [69]:
real_estate_stats = real_estate_mongo.describe()
real_estate_mean = real_estate_stats.mean()[0]

In [83]:
quartiles_real_estate = real_estate_mongo.quantile(q=[0.25,0.5,0.75])
quartiles_real_estate

,price
0.25,185000.0
0.50,285000.0
0.75,450250.0


In [95]:
quartiles_salary = salary_mongo.quantile(q=[0.25,0.50,0.75])
quartiles_salary

,salary_in_usd
0.25,43331.0
0.50,70912.0
0.75,102100.0


In [96]:
quartiles = [0.25,0.50,0.75]
real_estate_quartiles = []
salary_quartiles = []
for val in quartiles:
    real_estate_quartiles.append(quartiles_real_estate.loc[val][0])
    salary_quartiles.append(quartiles_salary.loc[val][0])

In [99]:
desired_money = []
for money in real_estate_quartiles:
    twenty_pct = money*0.2
    desired_money.append(twenty_pct)
desired_money

[37000.0, 57000.0, 90050.0]

In [100]:
results = {}
for salary in salary_quartiles:
    year_needed = []
    for down_payment in desired_money:
        years_to_earn = down_payment/salary
        year_needed.append(years_to_earn)
    results[str(salary)] = year_needed 

In [101]:
results

{'43331.0': [0.8538921326532967, 1.3154554476010247, 2.0781888255521452],
 '70912.0': [0.5217734657039711, 0.8038131768953068, 1.2698837996389891],
 '102100.0': [0.3623898139079334, 0.5582761998041136, 0.8819784524975515]}

## Taking this information to see how many years it would take to earn a 20% down payment on a house with this annual incoming ignoring taxes/other expenses

In [105]:
years_to_earn = pd.DataFrame(results)
years_to_earn = years_to_earn.rename(columns={'43331.0':'25th Percentile', '70912.0': '50th Percentile', '102100.0': '75th Percentile'})
years_to_earn

,25th Percentile,50th Percentile,75th Percentile
0,0.853892,0.521773,0.362390
1,1.315455,0.803813,0.558276
2,2.078189,1.269884,0.881978
